In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy
from pandas import *

In [3]:
class WumpusGame:
    def __init__(self):
        #this is the constructor
        self.createMaze()
        self.visited = list()
        self.blacklist = list()
        self.score = 0
        self.alive = True
        self.stuck_count = 0
        #self.down = [1,0]
        #self.right = [0,1]
        self.activated = list()
        self.gold_found = list()
        self.gold_f = False
        self.isSolvable = False
        self.path_found = False
        self.solved = False
        
        
        
        #while not self.isSolvable():
            #self.createMaze()  
    
        
            
    def createMaze(self):
        self.maze=list()
        self.ay=0
        self.ax=0
        for i in range(4):
            row=[]
            for j in range(4):
                row.append("[ ]")
            self.maze.append(row)
        self.maze[self.ax][self.ay]="[A]"
        
        stuff=["[W]","[P]","[P]","[P]", "[G]"]
        #Invalid places for objects
        invalidNodes = (0,0),(0,1),(1,0)
        for s in stuff:
            x=randint(4)
            y=randint(4)
            while self.maze[x][y]!="[ ]":
                x=randint(4)
                y=randint(4)
            if ((y,x)) not in invalidNodes:
                self.maze[x][y]=s
                
    def showMaze(self):
        S=""
        for i in range(4):
            for j in range(4):
                S+=self.maze[i][j]
            S+="\n"
        print(S)
        
    def validMove(self,position):
        valid = []
        x=position[0]
        y=position[1]

        up = [x-1,y]
        down = [x+1,y]
        right = [x,y+1]
        left = [x,y-1]

        valid.append(up)
        valid.append(down)
        valid.append(right)
        valid.append(left)

        return valid 
        
    #Normalising the QTable for each node, so that It always sums up to 1.
    #EX: [0.25 0.25 0.35 0.25] = [0.25 0.25 0.35 0.25] / 1.1
    def normaliseQTable(self,m1,m2,action,policy):
        policy[m1][m2] = policy[m1][m2]/policy[m1][m2].sum()
        
    #Function that updates the QTable (policy for the movements)
    #Here im adding the actual learningrate to the Qtable for the node.
    def updatePolicy(self,m1,m2,action,policy):
        policy[m1][m2][actions[i]] += learningRate
        self.normaliseQTable(m1,m2,action,policy)
        
    def applyPolicy(self,solution, explorationProb=0.0,useMax=False):
        holder = list()
        actions = list()
        current = [0,0]
        xm=[0,1,0,-1]
        ym=[1,0,-1,0]
        counter = 0
        X=[]
        Y=[]
        #while not current == "[G]":
        while not self.maze[current[0]][current[1]] == "[G]":
             #Random, in the start the explorationProbabilty will be high, so the majority of movements will be 
             #random. The exploration for new random paths that lead to the end.
            if numpy.random.random() < explorationProb:
                step = numpy.random.randint(4)
            
        #Using the Q table/Policy. 
        #Then when the explorationProbabilty gets lower and lower, we will almost only use the Q table to optimize the 
        #route the agent has taken.The more the training goes on, the Q table for each node will direct
        #the agent what movement to do to reach to goal faster, based on probability.
        #With this maze, we will reach a point where there is no faster way to the goal, 
        #than the Q table it has generated firstly with the random exploration, but later when the 
        #explorationProb is low, almost only with probability from the Q table.
            else:
                step = numpy.random.choice([0,1,2,3], p=policy[current[0]][current[1]])
                
                
            y,x = current
            
           
            holder.append(current)
            actions.append(step)
            
            x=x+xm[step]
            y=y+ym[step]
            if y < 0:
                y = 0
            if x < 0:
                x = 0
                
            if y > 3:
                y = 3
            if x > 3:
                x = 3
            valid = self.validMove(current)
            #print("CURRENT", current)
            #print(valid)
            if self.maze[y][x] == "[ ]" or self.maze[y][x] == "[G]":
                current = [y,x]
                if current not in valid:
                    pass
                    
                
                #print(current)
                
                #if current[0]<0:
                    #current[0] = 0
                #if current[0]==3:
                    #current[0] -= 1
                
                #if current[1]<0:
                    #current[1] = 0
                #if current[1]==3:
                    #current[1] -= 1
                X.append(x)
                Y.append(y)
            else:
                pass
                counter +=1
        holder.append(current)
        return holder,actions
    
    
    
    
    
    
    #BFS search, breadth first search
    def solvable(self,start):
        #Invalid search is nodes that are not movable when searching for a path and if the
        #maze is solvable or not.
        invalid_search = ["[P]","[W]"]
        #Making sure we are inside of the maze.
        if start[1] <= 3 and start[0] <= 3:
            #print(start[1], start[0])
            #print(f'0:{start[0]} 1:{start[1]}')
            
            #Generating neighbors for the current node.
                         #Right
            neighbors = ((start[0],     start[1] + 1),
                         #Down
                         (start[0] + 1, start[1]),
                         #Up
                         (start[0] - 1, start[1]),
                         #Left
                         (start[0],      start[1] - 1))
            
            
            # If we are at the end of the maze to the right
            if start[1] == 3:
                #neighbors = ((start[0],start[1]),(start[0] + 1, start[1]))
                neighbors = ((start[0],start[1]),(start[0] + 1, start[1]),(start[0] - 1 ,start[1]),(start[0], start[1] - 1) )
            
            # If we are at the bottom of the maze
            if start[0] == 3:
                #neighbors = ((start[0],start[1]),(start[0], start[1]))
                neighbors = ((start[0],start[1]),(start[0], start[1]),(start[0] - 1 ,start[1]), ((start[0], start[1] - 1)))
                
            # If we are at the top of the maze
            if start[0] == 0:
                neighbors = ((start[0],start[1] + 1),(start[0] + 1, start[1]),(start[0] ,start[1]), (start[0], start[1] - 1))
                
            #If we are at the end of the maze to the left
            if start[1] == 0:
                neighbors = ((start[0],start[1] + 1),(start[0] + 1, start[1]),(start[0] ,start[1]), (start[0], start[1]))

             
            #Iterating over the neighbors making sure we are inside of the maze,
            #and trying to jump on unwalkable nodes.
            #This loop will only walk on neighbor nodes that are walkable,
            #This means that if this function is able to find the gold, the maze is 
            #solvable.
            for neigh in neighbors:
                if neigh[0] > 3:
                    break
                if neigh[1] > 3:
                    break
                if neigh not in self.activated and self.maze[neigh[0]][neigh[1]] not in invalid_search:
                    self.activated.append(neigh)
                    if self.maze[neigh[0]][neigh[1]] == "[G]":
                        #print(neigh,self.maze[neigh[0]][neigh[1]])
                        #print("Maze is solvable")
                        self.gold_f = True
                        self.path_found = True
                        self.isSolvable = True
                    #Using recursions instead of while loop to call on the function with the new node.
                    self.solvable(neigh)
                    
        
        if self.isSolvable == True:
            return True
        else:
            return False


        
        
        
  
        

        

In [5]:
maze = WumpusGame()




canSolve = maze.solvable((0,0))


if canSolve == True:
    policy = numpy.ones((4,4,4))*0.25

    maze.showMaze()

    steps = []
    learningRate = 0.1
    for reps in range(1000):
        moves,actions = maze.applyPolicy(policy, explorationProb=1.0/(reps+1.0))
        
        #The actions are the actual moves that the agent has taken to reach the goal, and when the agent has 
        #reached the goal we reward it by updating the Q table. So after we have generated the random movement 
        #sequence that actually reaches the goal we are using this Q table/policy to optimize the solution.
        #Since the actions are generated from the steps and amount of steps the agent has taken, it will 
        #continue to optimize itself until it has found the best solution.
        for i in range(len(actions)):
            m1 = moves[i][0]
            m2 = moves[i][1]
            action=actions[i]
            #Rewarding the agent by updating the Qtable when it has reached the gold.
            #So for each node the agent has visited on the route to the goal, we are 
            #updating the Qtable for that specific node, by saying what the probability 
            #of taking an direction is to reach the reward. And when we later are iterating again,
            #we use this Qtable for the movements instead of random movements in the applyPolicy function.
            maze.updatePolicy(m1,m2,action,policy)
        print(reps, (len(actions)))
        steps.append(len(actions))


    path = []
    for a in moves:
        path.append(a)

    maze.showMaze()
    print(path)

else:
    maze.showMaze()
    print("Maze unsolvable")
        
        

        

        










[A][ ][ ][P]
[ ][ ][ ][G]
[ ][ ][ ][P]
[ ][ ][W][ ]

0 70
1 27
2 8
3 11
4 23
5 10
6 11
7 6
8 7
9 15
10 35
11 129
12 4
13 17
14 8
15 28
16 5
17 36
18 15
19 12
20 5
21 21
22 16
23 30
24 40
25 8
26 13
27 17
28 12
29 11
30 16
31 27
32 4
33 23
34 8
35 6
36 11
37 21
38 15
39 12
40 21
41 15
42 18
43 10
44 7
45 19
46 6
47 10
48 10
49 25
50 6
51 4
52 20
53 42
54 5
55 27
56 17
57 8
58 7
59 21
60 9
61 20
62 7
63 6
64 11
65 14
66 11
67 22
68 6
69 22
70 6
71 18
72 8
73 13
74 6
75 10
76 13
77 10
78 10
79 10
80 14
81 10
82 11
83 7
84 6
85 4
86 4
87 6
88 15
89 4
90 6
91 6
92 6
93 6
94 6
95 14
96 7
97 9
98 6
99 7
100 12
101 5
102 16
103 4
104 4
105 5
106 6
107 12
108 5
109 4
110 5
111 6
112 4
113 4
114 8
115 4
116 6
117 4
118 4
119 4
120 13
121 5
122 4
123 4
124 4
125 9
126 5
127 6
128 4
129 4
130 4
131 6
132 4
133 4
134 4
135 4
136 5
137 4
138 6
139 4
140 4
141 4
142 4
143 8
144 4
145 4
146 4
147 4
148 4
149 4
150 6
151 4
152 4
153 5
154 4
155 4
156 4
157 4
158 4
159 5
160 4
161 4
162 4
163 4
164 4
16